# Mixed language programming 

In [ ]:
!git clone https://github.com/jonaslindemann/compute-course-public.git

Cloning into 'compute-course-public'...
remote: Enumerating objects: 704, done.
remote: Counting objects: 100% (269/269), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 704 (delta 157), reused 241 (delta 136), pack-reused 435
Receiving objects: 100% (704/704), 49.08 MiB | 11.95 MiB/s, done.
Resolving deltas: 100% (201/201), done.
Checking out files: 100% (598/598), done.


In [ ]:
%cd compute-course-public/f2py

/content/compute-course-public/f2py


# Integrating Fortran and Python

* External code can be linked into Python using extension modules 
* Extension modules in Python uses a C Python API
* Can be used as normal Python modules
* Implementing a Python extension module is hard...

```C
#include "Python.h"

// The calculation function

static PyObject* sum(PyObject *self, PyObject *args)
{
    double a;
    double b;

    // Parse input arguments

    if (!PyArg_ParseTuple(args, "dd", &a, &b)) 
        return NULL;

    // Do our computation

    double c = a + b;

    // Return the results

    return Py_BuildValue("d", c);
}

// Module function table.

static PyMethodDef
module_functions[] = {
    { "sum", sum, METH_VARARGS, "Calculate sum." },
    { NULL }
};

// Module initialisation

void
initcext(void)
{
    Py_InitModule3("cext", module_functions, "A minimal module.");
}
```

# There is an easier way

* F2PY translates Fortran code and creates Python extension code
* Automatically passes Numpy Arrays as Fortran arrays
* Command line tool - Compiles and links modules automatically
* Hard to debug - Make sure existing code works before use

# Example 1 - simple.f90

```fortran
subroutine simple(a,b,c)

	real, intent(in) :: a, b
	real, intent(out) :: c

	c = a + b

end subroutine simple
```

Convert Fortran code to Python extension module using f2py

In [ ]:
!f2py -m simple -c simple.f90
%ls

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "simple" sources
f2py options: []
f2py:> /tmp/tmpfa95i0l7/src.linux-x86_64-3.7/simplemodule.c
creating /tmp/tmpfa95i0l7/src.linux-x86_64-3.7
Reading fortran codes...
	Reading file 'simple.f90' (format:free)
Post-processing...
	Block: simple
			Block: simple
Post-processing (stage 2)...
Building modules...
	Building module "simple"...
		Constructing wrapper function "simple"...
		  c = simple(a,b)
	Wrote C/API module "simple" to file "/tmp/tmpfa95i0l7/src.linux-x86_64-3.7/simplemodule.c"
  adding '/tmp/tmpfa95i0l7/src.linux-x86_64-3.7/fortranobject.c' to sources.
  adding '/tmp/tmpfa95i0l7/src.linux-x86_64-3.7' to include_dirs.
copying /usr/local/lib/python3.7/dist-packages/numpy/f2py/src/fortranobject.c -> /tmp/tmpfa95i0l7/s

Import the module in Python and print documentation

In [ ]:
import simple
print(simple.__doc__)

This module 'simple' is auto-generated with f2py (version:1.21.6).
Functions:
  c = simple(a,b)
.


In [ ]:
print(simple.simple.__doc__)

c = simple(a,b)

Wrapper for ``simple``.

Parameters
----------
a : input float
b : input float

Returns
-------
c : float



Execute code in the generated extension module.

In [ ]:
a = 42
b = 42
c = simple.simple(a, b)
print(c)

84.0


# Example 2 - arr1.f90

In this example we will use a Fortran compiled extension module for multiplying matrices.

```fortran
! A[r,s] * B[s,t] = C[r,t]
subroutine matrix_multiply(A,r,s,B,t,C)
	integer :: r, s, t
	real, intent(in) :: A(r,s)
	real, intent(in) :: B(s,t)
	real, intent(out) :: C(r,t)

	C = matmul(A,B)
end subroutine matrix_multiply
```

In [ ]:
!f2py -m arr1 -c arr1.f90

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "arr1" sources
f2py options: []
f2py:> /tmp/tmpcye8zcnf/src.linux-x86_64-3.7/arr1module.c
creating /tmp/tmpcye8zcnf/src.linux-x86_64-3.7
Reading fortran codes...
	Reading file 'arr1.f90' (format:free)
Post-processing...
	Block: arr1
			Block: matrix_multiply
Post-processing (stage 2)...
Building modules...
	Building module "arr1"...
		Constructing wrapper function "matrix_multiply"...
		  c = matrix_multiply(a,b,[r,s,t])
	Wrote C/API module "arr1" to file "/tmp/tmpcye8zcnf/src.linux-x86_64-3.7/arr1module.c"
  adding '/tmp/tmpcye8zcnf/src.linux-x86_64-3.7/fortranobject.c' to sources.
  adding '/tmp/tmpcye8zcnf/src.linux-x86_64-3.7' to include_dirs.
copying /usr/local/lib/python3.7/dist-packages/numpy/f2py/src/fortranobject.c 

In [ ]:
import arr1
print(arr1.__doc__)

This module 'arr1' is auto-generated with f2py (version:1.21.6).
Functions:
  c = matrix_multiply(a,b,r=shape(a,0),s=shape(a,1),t=shape(b,1))
.


In [ ]:
print(arr1.matrix_multiply.__doc__)

c = matrix_multiply(a,b,[r,s,t])

Wrapper for ``matrix_multiply``.

Parameters
----------
a : input rank-2 array('f') with bounds (r,s)
b : input rank-2 array('f') with bounds (s,t)

Other Parameters
----------------
r : input int, optional
    Default: shape(a,0)
s : input int, optional
    Default: shape(a,1)
t : input int, optional
    Default: shape(b,1)

Returns
-------
c : rank-2 array('f') with bounds (r,t)



In [ ]:
import numpy as np

# --- order='F' is important as Fortran stores arrays
# --- columnwise and NumPy by default stores them row wise as
# --- in C

A = np.ones((6,6), 'f', order='F') * 10.0
B = np.ones((6,6), 'f', order='F') * 20.0
C = np.zeros((6,6), 'f', order='F')

print("id of C before multiply =",id(C))

C = arr1.matrix_multiply(A, B)

print("id of C after multiply =",id(C))

print(C)

id of C before multiply = 139866421133360
id of C after multiply = 139866538889648
[[1200. 1200. 1200. 1200. 1200. 1200.]
 [1200. 1200. 1200. 1200. 1200. 1200.]
 [1200. 1200. 1200. 1200. 1200. 1200.]
 [1200. 1200. 1200. 1200. 1200. 1200.]
 [1200. 1200. 1200. 1200. 1200. 1200.]
 [1200. 1200. 1200. 1200. 1200. 1200.]]


Notice that the id or memory references of C changes before and after the call to our Fortran extension. This is due to the fact that the left assignment will create a new NumPy array reference and data will be copied from Fortran to the NumPy array. 

In the next example we will show how to avoid this.

# Example 3 - arra2.f90

In this example we use the variable attribute, intent(inout), to indicate that the array will be passed by reference to the Fortran extension module and modified in place. This requires the NumPy array to be created with the order='F' option.

```fortran
! A[r,s] * B[s,t] = C[r,t]
subroutine matrix_multiply2(A,r,s,B,t,C)
	integer :: r, s, t
	real, intent(in) :: A(r,s)
	real, intent(in) :: B(s,t)
	real, intent(inout) :: C(r,t)

	C = matmul(A,B)
end subroutine matrix_multiply2
```

In [ ]:
!f2py -m arr2 -c arr2.f90

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "arr2" sources
f2py options: []
f2py:> /tmp/tmp64c3a3_g/src.linux-x86_64-3.7/arr2module.c
creating /tmp/tmp64c3a3_g/src.linux-x86_64-3.7
Reading fortran codes...
	Reading file 'arr2.f90' (format:free)
Post-processing...
	Block: arr2
			Block: matrix_multiply2
Post-processing (stage 2)...
Building modules...
	Building module "arr2"...
		Constructing wrapper function "matrix_multiply2"...
		  matrix_multiply2(a,b,c,[r,s,t])
	Wrote C/API module "arr2" to file "/tmp/tmp64c3a3_g/src.linux-x86_64-3.7/arr2module.c"
  adding '/tmp/tmp64c3a3_g/src.linux-x86_64-3.7/fortranobject.c' to sources.
  adding '/tmp/tmp64c3a3_g/src.linux-x86_64-3.7' to include_dirs.
copying /usr/local/lib/python3.7/dist-packages/numpy/f2py/src/fortranobject.c

In [ ]:
import arr2
print(arr2.__doc__)

This module 'arr2' is auto-generated with f2py (version:1.21.6).
Functions:
  matrix_multiply2(a,b,c,r=shape(a,0),s=shape(a,1),t=shape(b,1))
.


In [ ]:
print(arr2.matrix_multiply2.__doc__)

matrix_multiply2(a,b,c,[r,s,t])

Wrapper for ``matrix_multiply2``.

Parameters
----------
a : input rank-2 array('f') with bounds (r,s)
b : input rank-2 array('f') with bounds (s,t)
c : in/output rank-2 array('f') with bounds (r,t)

Other Parameters
----------------
r : input int, optional
    Default: shape(a,0)
s : input int, optional
    Default: shape(a,1)
t : input int, optional
    Default: shape(b,1)



In [ ]:
A = np.ones((6,6), 'f', order='F') * 10.0
B = np.ones((6,6), 'f', order='F') * 20.0
C = np.zeros((6,6), 'f', order='F')

print("id of C before multiply =",id(C))

arr2.matrix_multiply2(A, B, C)

print("id of C after multiply =",id(C))

print(C)

id of C before multiply = 139866421235408
id of C after multiply = 139866421235408
[[1200. 1200. 1200. 1200. 1200. 1200.]
 [1200. 1200. 1200. 1200. 1200. 1200.]
 [1200. 1200. 1200. 1200. 1200. 1200.]
 [1200. 1200. 1200. 1200. 1200. 1200.]
 [1200. 1200. 1200. 1200. 1200. 1200.]
 [1200. 1200. 1200. 1200. 1200. 1200.]]


No we see that the id or references of the C array are the same before and after the call to our Fortran extension. The Fortran extension now got a direct reference to the memory of the NumPy array. This is the most efficient way of communicating with Fortran code as there will be no copies made. Copying large matrices can consume a lot of memory and CPU time.

# Example 4 - matrix.f90

In this example we will use the Fortran matmul function for multiplying matrices.

```fortran
module matrix

contains

! A[r,s] * B[s,t] = C[r,t]
subroutine matrix_multiply(A,r,s,B,t,C)
	integer :: r, s, t
	real, intent(in) :: A(r,s)
	real, intent(in) :: B(s,t)
	real, intent(out) :: C(r,t)

	C = matmul(A,B)
end subroutine matrix_multiply

! A[r,s] * B[s,t] = C[r,t]
subroutine matrix_multiply2(A,r,s,B,t,C)
        integer :: r, s, t
        real, intent(in) :: A(r,s)
        real, intent(in) :: B(s,t)
        real, intent(inout) :: C(r,t)

        C = matmul(A,B)
end subroutine matrix_multiply2

end module matrix
```

In [ ]:
!f2py -m matrix -c matrix.f90

running build
running config_cc
unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
build_src
building extension "matrix" sources
f2py options: []
f2py:> /tmp/tmpngh8_7w0/src.linux-x86_64-3.7/matrixmodule.c
creating /tmp/tmpngh8_7w0/src.linux-x86_64-3.7
Reading fortran codes...
	Reading file 'matrix.f90' (format:free)
Post-processing...
	Block: matrix
			Block: matrix
				Block: matrix_multiply
				Block: matrix_multiply2
Post-processing (stage 2)...
	Block: matrix
		Block: unknown_interface
			Block: matrix
				Block: matrix_multiply
				Block: matrix_multiply2
Building modules...
	Building module "matrix"...
		Constructing F90 module support for "matrix"...
			Constructing wrapper function "matrix.matrix_multiply"...
			  c = matrix_multiply(a,b,[r,s,t])
			Constructing wrapper function "matrix.matrix_multiply2"...
			  matrix_multip

In [ ]:
import matrix
print(matrix.__doc__)

This module 'matrix' is auto-generated with f2py (version:1.21.6).
Functions:
Fortran 90/95 modules:
  matrix --- matrix_multiply(),matrix_multiply2().


In [ ]:
print(matrix.matrix.__doc__)

c = matrix_multiply(a,b,[r,s,t])

Wrapper for ``matrix_multiply``.

Parameters
----------
a : input rank-2 array('f') with bounds (r,s)
b : input rank-2 array('f') with bounds (s,t)

Other Parameters
----------------
r : input int, optional
    Default: shape(a,0)
s : input int, optional
    Default: shape(a,1)
t : input int, optional
    Default: shape(b,1)

Returns
-------
c : rank-2 array('f') with bounds (r,t)
matrix_multiply2(a,b,c,[r,s,t])

Wrapper for ``matrix_multiply2``.

Parameters
----------
a : input rank-2 array('f') with bounds (r,s)
b : input rank-2 array('f') with bounds (s,t)
c : in/output rank-2 array('f') with bounds (r,t)

Other Parameters
----------------
r : input int, optional
    Default: shape(a,0)
s : input int, optional
    Default: shape(a,1)
t : input int, optional
    Default: shape(b,1)



Modules generated by f2py are often better to import by using the from * import statement

In [ ]:
del(matrix) # Removes the matrix module

In [ ]:
from matrix import *

In [ ]:
matrix.matrix_multiply2(A, B, C)
print(C)

[[1200. 1200. 1200. 1200. 1200. 1200.]
 [1200. 1200. 1200. 1200. 1200. 1200.]
 [1200. 1200. 1200. 1200. 1200. 1200.]
 [1200. 1200. 1200. 1200. 1200. 1200.]
 [1200. 1200. 1200. 1200. 1200. 1200.]
 [1200. 1200. 1200. 1200. 1200. 1200.]]


Now the Fortran modules names can be used directly withour the additional matrix prefix.